In [ ]:
#  PIP install time 
#  PIP install oci
#  PIP install os

In [ ]:
# Config.txt file
[DEFAULT]
user= <your-ociid>
fingerprint=<your-fingerprint>
key_file=<path-to-private-key-file>
tenancy=<ocid1.tenancy>
region=<region-1>

In [ ]:
import time
import oci
import os 

# Set your parameters here
COMPARTMENT_ID = os.environ.get('COMPARTEMENT_ID')
IMAGE_ID = os.environ.get('IMAGE_ID')
SUBNET_ID = os.environ.get('SUBNET_ID')
SHAPE = os.environ.get('SHAPE')

# List of availability domains
ADs = ["AD-1", "AD-2", "AD-3"]

# Create an instance of the OCI ComputeClient
config = oci.config.from_file(file_location=os.environ.get('CONFIG')) #Get configuration file , config.txt
compute_client = oci.core.ComputeClient(config)

# Start an infinite loop
while True:
    # Loop over availability domains
    for AD in ADs:
        print(f"Attempting to create instance in {AD}...")

        try:
            # Attempt to create the instance
            launch_instance_response = compute_client.launch_instance(
                oci.core.models.LaunchInstanceDetails(
                    compartment_id=COMPARTMENT_ID,
                    availability_domain=AD,
                    shape=SHAPE,
                    image_id=IMAGE_ID,
                    subnet_id=SUBNET_ID
                )
            )

            # Check the status of the instance
            if launch_instance_response.status == 200:
                print(f"Instance created successfully in {AD}.")
                exit(0)
            else:
                print(f"Failed to create instance in {AD}. Trying next availability domain...")
        except oci.exceptions.ServiceError as e:
            error_message = f"Failed to create instance in {AD}."

            # Retrieve the error details
            if e.status:
                error_message += f" HTTP status code: {e.status}"
            if e.headers and 'opc-request-id' in e.headers:
                error_message += f" Request ID: {e.headers['opc-request-id']}"
            if e.code:
                error_message += f" Error code: {e.code}"
            if e.message:
                error_message += f" Error message: {e.message}"
            if e.request_id:
                error_message += f" Service request ID: {e.request_id}"

            print(error_message)

    # If the script reaches this point, all attempts have failed
    print("Failed to create instance in all availability domains. Waiting 1 minute before trying again...")
    # Wait for 1 minute before the next attempt
    time.sleep(120)
